# I used a lot of the code cells from this link in order to build this. 

https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/langchain-retrieval-agent.ipynb#scrollTo=wXi_0ipTvM_l

In [1]:
import openai
import langchain
import langchain_pinecone
import pinecone
import os
import getpass

c:\Users\tempj\anaconda3\envs\wiCodeAgentEnv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from pinecone import Pinecone, ServerlessSpec
use_serverless = True
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)




In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [4]:
# Initializing pinecone client and specify index connection

pc_api_key = os.environ.get('PINECONE_API_KEY')
pc_index_name = os.environ.get('PINECONE_INDEX_NAME')

pc = Pinecone(api_key=pc_api_key)
index = pc.Index(pc_index_name)

In [5]:
# Initialize the OpenAI client

api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)


I had been getting errors initializing the embedding object. The langchain bot told me how to update it

The error message you received is related to the model_name argument in the OpenAIEmbeddings constructor.
In recent versions of Langchain, the model_name argument has been moved to the model_kwargs dictionary for consistency with other embedding classes.
To fix the error, you can either:

• Move the model_name argument to the model_kwargs dictionary:
• Use the model argument instead of model_name:


In [ ]:
# embeddings = OpenAIEmbeddings(
#     model_kwargs={
#         "model_name": "text-embedding-ada-002"
#     }
# )

# later on, you can add the model_kwargs argument
# embeddings.model_kwargs = {
#     "max_length": 8192,
#     "engine": "davinci",
#     "temperature": 0.1,
#     "top_p": 1.0,
#     "n": 1
# }

In [6]:
# Create an openAI embedding object

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

Yes, you're on the right track! If you already have a knowledge base built in Pinecone, you can use Pinecone.from_existing_index(index_name) to connect to your existing Pinecone index and create a Pinecone object.
Once you have the Pinecone object, you can create a retriever using Pinecone.as_retriever() method.
Here's an example of how you can use the Pinecone object to create a RetrievalQA object:

In [7]:
# Creating our embedding object for queries from a specific index

from langchain.vectorstores import Pinecone

pinecone = Pinecone.from_existing_index(
    index_name=pc_index_name,
    embedding=embeddings
)

vectorstore = pinecone

In [8]:
# LangChain defines a Retriever interface which wraps an index that can return relevant Documents given a string query.
# We initialize a retriever object from our vectorstore object

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
retriever.invoke("What's the live load for garage floors?")

In [10]:
# chat completion llm
llm = ChatOpenAI(
    openai_api_key=api_key, #may be redundant if we have api key in .env file
    model_name='gpt-4-0125-preview',
    temperature=0.0
)

In [11]:
from langchain_core.prompts import ChatPromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.

{context}


Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
for chunk in rag_chain.stream("What's the live load for garage floors?"):
    print(chunk, end="", flush=True)

The live load for garage floors is 50 pounds per square foot.